## 小象学院课程
## Pyhton数据分析
## 美国化学学会杂志-1
“美国化学学会杂志”是美国化学学会出版的首要期刊，也是所有化学领域排名最高的期刊之一。本项目的数据集是该期刊从1996到2016的论文集,用以分析作者发表论文的情况。

> **提示**：<br>
1、这样的文字将会指导你如何使用 jupyter Notebook 来完成项目。<br>
2、你可以通过单击代码区域，然后使用键盘快捷键 Shift+Enter 或 Shift+Return 来运行代码。或者在选择代码后使用执行（run cell）按钮执行代码。Markdown的文字区域也同样可以如此操作。<br>
3、在如下有# TODO 提示的地方，将代码补全，实现注释中所要求的功能。<br>
4、在有"** 回答：**" 提示的地方，回答其上所提出的问题。<br>


### 分析流程
这里对本项目分析的具体流程和内容进行了概述，以对本项目有整体的了解<br>
1 导入数据，分析表结构：导入sqlite数据文件，创建数据库，以及对数据库中的表结构进行分析，以对本项目的数据集有基本的认识。<br>
2 计算论文的数量：计算每个作者对应的论文数量，得到论文数据量大于20的作者发表的论文数量，以对论文数量大于20的作者发表的论文情况进行研究<br>
3 获得作者列表：得到论文数量大于20的作者，以了解论文数量大于20的作者有哪些<br>
4 合并数据、存入文件：通过以上分析，将得到作者和作者发表的论文数量的数据进行合并，即将作者、作者ID号、论文数量进行合并，将数据存入JSON文件<br>

### 项目数据集介绍：
数据库对于数据分析非常重要，它可以存储大量有某种关系的表，通过对表进行连接操作，以实现对数据的潜在分析。
本项目采用的数据集以sqlite的方式进行存储，数据集中包括三个表：
* Authors：对作者进行描述
* Paper_Authors：对作者发表论文的情况进行描述
* Papers：关于论文的表

### 导入数据，分析表结构

导入必备的Python库

In [2]:
import sqlite3
import json
import numpy as np

### 连接数据库和得到数据库中的表

获得数据库的连接，游标

In [3]:
def connect_sqlite(sqlite_path):
    # TODO
    # 连接数据库，得到游标，为了使用游标对表按照字段名获取列数据，
    # 这里需要将conn.row_factory设置为sqlite3.Row,这个设置很重要，不然不能通过字段名获得数据
    
    conn = sqlite3.connect(sqlite_path)
    conn.row_factory=sqlite3.Row
    cur = conn.cursor()
    
    return conn, cur

conn, cur = connect_sqlite("./dataFile/database.sqlite")

得到数据库中的所有表

In [4]:
# TODO
# 打印数据库中的所有表的名字
table_names=cur.execute("select name from sqlite_master where type='table';").fetchall()
for table_name in table_names:
    print(table_name['name'])

Authors
Paper_Authors
Papers


### 获得Authors表的字段名,及表结构信息
获取Authors表的列名

In [5]:
# 调用游标的description方法，获得Authors表的字段
cur.execute("select * from Authors")
col_name_list = [tuple[0] for tuple in cur.description]
print("Authors表的字段名为：",col_name_list)

Authors表的字段名为： ['authorID', 'title', 'forename', 'initials', 'surname', 'orcidID']


####  本项目主要对Authors表的authorID、forename、surname字段进行分析
* 字段名的说明：<br>
authorID：作者的ID号<br>
surname：姓<br>
forename：名字<br>

获取Authors表结构的信息

In [6]:
# 执行PRAGMA table_info语句获取Authors表结构信息，并且打印
cur.execute("PRAGMA table_info({})".format('Authors'))#结果集将为每个包含在给出列序列的索引、表格内的列索引、列名称的列显示一行。
for author in cur.fetchall():
    print([author[i] for i in range(len(author))])

[0, 'authorID', 'INTEGER', 0, None, 1]
[1, 'title', 'TEXT', 0, None, 0]
[2, 'forename', 'TEXT', 0, None, 0]
[3, 'initials', 'TEXT', 0, None, 0]
[4, 'surname', 'TEXT', 0, None, 0]
[5, 'orcidID', 'TEXT', 0, None, 0]


查看Authors表中author的数量，且是否唯一

In [7]:
cur.execute("select * from Authors")
# TODO
# 获得Authors表中authorID的个数及非重复个数，其中，使用np.unique，可以计算非重复个数
rows=cur.fetchall()
authorlist=[]
for row in rows:
    authorlist.append(row['authorID'])    
authors_all = len(authorlist)
authors_only = len(np.unique(authorlist))
print(authors_all)
print(authors_only)    

120670
120670


#### 问题：Authors表中有多少个author，唯一吗

** 回答： 120670,唯一

### 获得Papers表的信息

In [8]:
cur.execute("select * from Papers")
# TODO
# 获取Papers表的所有字段名，以列表形式进行输出
name_paper_lst=[i[0] for i in cur.description]    
print(name_paper_lst)

['paperID', 'DOI', 'type', 'title', 'abstract', 'volume', 'issue', 'startpg', 'endpg', 'received', 'publishedOnline', 'publishedPrint', 'authorChoiceOA', 'editorChoiceOA', 'views', 'citations', 'scrapeDT']


#### 下一作业中会涉及到对Papers表中的'paperID'、'views'，'publishedOnline'字段进行分析
* 字段名的说明：<br>
paperID：paper的编号<br>
type：文献所属的类型<br>
title：标题<br>
abstract：摘要<br>
startpg：起始页<br>
publishedOnline：发表日期<br>
views：被查看次数<br>
citations：引用次数<br>

In [9]:
print("以下是Papers表中paperID，publishedOnline，views字段的前10行数据\n")
# TODO
# 调用fetchmany方法获得前十行数据，调用字段名，打印Papers表中paperID，publishedOnline，views字段的数据,要求paperID号从1开始
cur.execute("select * from Papers;")
items=cur.fetchmany(10)
for item in items:
    print(item['paperID'],item['publishedOnline'],item['views'])

以下是Papers表中paperID，publishedOnline，views字段的前10行数据

1 1996-01-10 00:00:00.000000 1872
2 1996-01-10 00:00:00.000000 979
3 1996-01-10 00:00:00.000000 481
4 1996-01-10 00:00:00.000000 456
5 1996-01-10 00:00:00.000000 769
6 1996-01-10 00:00:00.000000 3672
7 1996-01-10 00:00:00.000000 595
8 1996-01-10 00:00:00.000000 639
9 1996-01-10 00:00:00.000000 564
10 1996-01-10 00:00:00.000000 225


查看Papers表的paperID是否唯一

In [10]:
cur.execute("select * from Papers")
# TODO
# 获得Papers表中paperID的个数及非重复个数
paperids=cur.fetchall()
paperlist=[]
for paperid in paperids:
    paperlist.append(paperid['paperID'])
paper_all = len(paperlist)
paper_only = len(np.unique(paperlist))
print(paper_all)
print(paper_only)

59467
59467


问题：Papers表中有多少论文，paperID是唯一的吗

** 回答： 59467,唯一

### 获得Paper_Authors表的字段名,以及表信息

查询Paper_Authors表的前二十行数据

In [11]:
cur.execute("select * from Paper_Authors limit 20")

In [12]:
# TODO
# 查看Paper_Authors表包含的字段
paper_Authors_list = [tuple[0] for tuple in cur.description]
print(paper_Authors_list)

['paperID', 'authorID']


字段说明：<br>
paperID：paper的ID,对应了Papers表中的paperID<br>
authorID: author的ID,对应了Authors表中的authorID

In [13]:
print("以下是Paper_Authors表中authorID，paperID字段的前20行数据\n")
# TODO
# 调用字段名，打印Paper_Authors表中paperID，authorID字段的前20行数据
paper_Authors=cur.fetchall()
for i in paper_Authors:
    print(i['paperID'],i['authorID'])

以下是Paper_Authors表中authorID，paperID字段的前20行数据

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
2 9
2 10
2 11
3 12
3 13
3 14
4 15
4 16
4 17
4 18
4 19
5 20


问题：从以上结果得出什么结论<br>

** 回答： ** <br>

分析Authors、Papers、Paper_Authors表的数据量存在的隐含关系

In [14]:
cur.execute("select * from Paper_Authors")
paper_author_data = cur.fetchall()

In [15]:
paper_author_authorid = [paper_author["authorID"] for paper_author in paper_author_data]
print("Paper_Authors表包括: {}条数据".format(len(paper_author_authorid)))
print("Paper_Authors表包括: {}条唯一的authorID数据".format(len(np.unique(paper_author_authorid))))

Paper_Authors表包括: 278639条数据
Paper_Authors表包括: 120670条唯一的authorID数据


In [16]:
paper_author_paperid = [paper_author["paperID"] for paper_author in paper_author_data]
print("Paper_Authors表包括: {}条数据".format(len(paper_author_paperid)))
print("Paper_Authors表包括: {}条唯一的paperID数据".format(len(np.unique(paper_author_paperid))))

Paper_Authors表包括: 278639条数据
Paper_Authors表包括: 58932条唯一的paperID数据


问题：<br>
1、Paper_Authors表共有多少条数据<br>
2、Authors表和Paper_Authors表中的作者数量一样吗，说明了什么?<br>
3、Papers表和Paper_Authors表中的paper数量一样吗，说明了什么?<br>

** 回答： ** <br>总共有278639条数据
1、总共有278639条数据<br>
2、一样，说明了Authors表中的论文作者和这张表中的作者一样<br>
3、59467>58932说明paper_Authors中的作者发表的部分论文并没有被记录在这张表里面

### 计算论文的数量

对项目的数据集进行基本了解之后，下面我们将通过Authors表查询前100名的作者(由于作者太多会增加运行时间，所以本项目只取了前100名作者的数据进行分析)，依据作者的authorID在Paper_Authors表中查询对应的数据，通过Paper_Authors表中的paperID字段，得到该作者发表的所有paperID号，进而对前100名作者发表的论文数量进行统计。<br>
* 发表的论文数量进行统计，得到论文数大于20的作者的论文数

In [18]:
def get_authorid_and_papernum(authors):
    """
    功能：
        计算论文数量大于20的作者，得到该作者的authorID和论文数量
    参数：
        authors是Authors表的每一行数据
    返回值：
        以列表的形式返回authorID和论文数量
    """
    paper_authors=[]
    # TODO
    # 通过Paper_Authors表的authorID，得到每个作者发表的论文数量，计算论文数量大于20的作者，得到该作者的authorID和论文数量，
    # 将authorID和论文数量用paper_authors列表存放，paer_authors的格式如 paper_authors = [[1,56],[2,23]],表示authorID==1的作者，
    # 发表了56个论文，authorID==2的作者，发表了23个论文
    list3=[]
    list4=[]
    list6=[]
    list1=cur.execute("select authors.authorID,paperID from authors left outer join Paper_Authors on authors.authorID=Paper_Authors.authorID;").fetchall()
    for i in list1:
        list2=[i[0],i[1]]
        list3.append(list2)
    for i in range(1,101):
        num=0
        for item in list3:
            if item[0]==i:
                num+=1
            else:
                continue
        list4.append(num)
    list5=[i for i in list4 if i!=0]
    for j in authors:
        list6.append(j['authorID'])
    list7=[list(i) for i in list(zip(list6,list5))]
    paper_authors=[i for i in list7 if i[1]>20]
    return paper_authors

# 由于Authors的数据较多，因此，这里仅获得前100名author的authorID，对前100名作者的论文情况进行分析
authors = cur.execute("select * from Authors limit 100").fetchall()
authorid_papernum_lst = get_authorid_and_papernum(authors)
print(authorid_papernum_lst)

[[5, 30], [18, 75], [20, 27], [22, 23], [26, 27], [29, 21], [31, 43], [48, 46], [53, 40], [57, 121], [64, 93], [65, 58], [69, 39], [70, 72], [73, 137], [74, 117], [79, 79], [89, 25], [91, 45], [92, 40]]


从以上的结果中进行分析，得到作者ID号列表，列表中存放符合条件的authorID号，以后会经常用到此作者列表，比如分析作者的名字列表<br>
正确答案：作者ID号列表中的authorID号为[5, 18, 20, 22, 26, 29, 31, 48, 53, 57, 64, 65, 69, 70, 73, 74, 79, 89, 91, 92]，作者列表包含20个作者的ID号

In [81]:
# TODO
# 从以上得到的结果authorid_papernum_lst列表中得到作者的ID号列表
authors_id = [i[0] for i in authorid_papernum_lst]

print("作者的ID号列表中的authorID号为：{},作者列表包含{}个作者的ID号".format(authors_id,len(authors_id)))

作者的ID号列表中的authorID号为：[5, 18, 20, 22, 26, 29, 31, 48, 53, 57, 64, 65, 69, 70, 73, 74, 79, 89, 91, 92],作者列表包含20个作者的ID号


### 得到作者的名字列表
通过对上一个问题进行分析，我们得到了论文数量和作者的ID号列表，现在需要根据作者的ID号列表authors_id，在Authors表中查询每个作者的forename和surname，进而得到作者的名字列表。<br>
名字=名+姓(forename=名，surname=姓)<br>

In [101]:
def get_author_name(cur,author_id):
    """
    功能：
        根据authorID，从Authors表中得到forename和surname字段，进而得到作者的名字
    参数：
        cur是游标，author_id是作者的authorID号
    返回值：
        返回作者的名字
    """
    authors_forename=[]
    items = cur.execute("select forename,surname from Authors where authorID={};".format(author_id)).fetchall()
    # TODO
    # 使用单空格将Authors表中的forename和surname字段进行拼接，返回作者的名字
    for item in items:
        name=item['forename']+' '+item['surname']
        authors_forename.append(name) 

    return authors_forename

# authors_name_lst是作者的名字列表，该列表包括20个名字
authors_name_lst = [get_author_name(cur, author_id)[0] for author_id in authors_id]
print("作者的名字列表为：{}".format(authors_name_lst))

作者的名字列表为：['James Snyder', 'Kenneth Karlin', 'Per-Ola Norrby', 'K. Sharpless', 'M. Ghadiri', 'Kurt Gothelf', 'Karl Jørgensen', 'Ivano Bertini', 'Claudio Luchinat', 'Michael Wasielewski', 'Robert Grubbs', 'Joseph Ziller', 'Jay Winkler', 'Harry Gray', 'Stephen Lippard', 'Brian Hoffman', 'Atsuhiro Osuka', 'Fernando Bernardi', 'Massimo Olivucci', 'Michael Robb']


### 合并数据，存入文件

通过以上的分析，我们得到了作者的ID号列表，以及对应的作者名字列表和作者的论文数，现在我们要将这些数据进行合并，指在将合并后的数据存入文件进行保存。

合并数据

In [118]:
total_author_paper_lst=[]
l1=[tuple(i) for i in authorid_papernum_lst]
d1=dict(zip(authors_name_lst,l1))
# TODO
# 按照authorID的ID号，将得到的数据列表author_name_lst(authors_name)、authorid_papernum_lst(author_id,papers_num)依次赋值给
# 名称为"author_name","author_id","papers_num"的数据结构，比如字典，然后在for循环中将数据进行打印，最后将结果存入total_author_paper_lst中
total_author_paper_lst.append(d1)
print(total_author_paper_lst)



[{'James Snyder': (5, 30), 'Kenneth Karlin': (18, 75), 'Per-Ola Norrby': (20, 27), 'K. Sharpless': (22, 23), 'M. Ghadiri': (26, 27), 'Kurt Gothelf': (29, 21), 'Karl Jørgensen': (31, 43), 'Ivano Bertini': (48, 46), 'Claudio Luchinat': (53, 40), 'Michael Wasielewski': (57, 121), 'Robert Grubbs': (64, 93), 'Joseph Ziller': (65, 58), 'Jay Winkler': (69, 39), 'Harry Gray': (70, 72), 'Stephen Lippard': (73, 137), 'Brian Hoffman': (74, 117), 'Atsuhiro Osuka': (79, 79), 'Fernando Bernardi': (89, 25), 'Massimo Olivucci': (91, 45), 'Michael Robb': (92, 40)}]


存入文件

In [116]:
file_path = './dataFile/author_paper_data.json'
# TODO
# 将数据total_author_paper_lst存入author_paper_data.json文件
json_str=json.dumps(d1,ensure_ascii=False)
with open(file_path,'wb')as f:
    f.write(json_str.encode('utf-8'))

关闭数据库

In [119]:
# TODO
# 提交事务
# 关闭数据库
conn.commit()
conn.close()


### 你好，首先恭喜你完成了以上作业，下面我们将开启下一个旅程，去完成第二个作业，第二个作业是依据本作业存入的数据，进一步对该项目进行分析，内容如下：
* 依据作者ID号列表，对每个作者的发表论文的浏览次数进行分析
* 依据作者ID号列表，对每个作者发表论文的日期进行分析